In [1]:
from setfit import SetFitModel, sample_dataset
import pandas as pd
from datasets import load_dataset
import numpy as np
import torch

/home/xiyan/data/conda/envs/GPU/lib/python3.11/site-packages/transformers/utils/generic.py:462: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/xiyan/data/conda/envs/GPU/lib/python3.11/site-packages/transformers/utils/generic.py:319: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


### Data preperation and model training

In [45]:
dataset = pd.read_csv("scratch/3_SML_benchmark.csv")

In [46]:
dataset.tail()

,Unnamed: 0,index,filename,text_original,text_clean,date,year,month,outlet,country,south,RA
475,475,101008,newsarticle/Toronto Star/Neutral & Urgent 2006...,\n Business\nTime to invest in foreign stude...,Business Time to invest in foreign students Da...,2022-09-10,2022,9,Toronto Star,Canada,0,0
476,476,100986,newsarticle/Toronto Star/Neutral & Urgent 2006...,\n Politics\nMeet the Greens who might take ...,Politics Meet the Greens who might take a run ...,2022-07-04,2022,7,Toronto Star,Canada,0,0
477,477,2962,A cautionary flag for oil development;Study ca...,"\nMarch 1, 2020 Sunday\n\n\nCopyright 2020 Tor...","March 1, 2020 Sunday Copyright 2020 Toronto St...",2020-03-01,2020,3,Toronto Star,Canada,0,0
478,478,74551,The Toronto Star_2018-8-68.txt,""""";""x""\n""1"";"" 82\n\n\n ...",""""";""x"" ""1"";"" 82 The Toronto Star August 5, 201...",2018-08-01,2018,8,Toronto Star,Canada,0,1
479,479,72479,The Toronto Star_2011-7-21_39703.txt,""""";""x""\n""1"";""DATE: 21.7.2011\nHEADLINE: Securi...",""""";""x"" ""1"";""DATE: 21.7.2011 HEADLINE: Security...",2011-07-01,2011,7,Toronto Star,Canada,0,1


In [47]:
df = dataset[["text_clean", "RA"]]
df.rename(columns={'RA': 'label','text_clean':'text'}, inplace=True)

/sctmp/xiyan/ipykernel_2848872/675981800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'RA': 'label','text_clean':'text'}, inplace=True)


In [63]:
#train = df.sample(frac = 0.7)
train = df.groupby('label').sample(n=150)
df2 = df.drop(train.index)
validation = df2.sample(frac = 0.5)
test = df2.drop(validation.index)

In [64]:
train.to_csv("scratch/train.csv")
validation.to_csv("scratch/validation.csv")
test.to_csv("scratch/test.csv")

In [65]:
print(len(train),len(validation),len(test))

300 90 90


In [3]:
dataset = load_dataset("csv", data_files={'train':'scratch/train.csv','validation':'scratch/validation.csv','test': 'scratch/test.csv'})

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 300
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 90
    })
    test: Dataset({
        features: ['Unnamed: 0', 'text', 'label'],
        num_rows: 90
    })
})

In [5]:
#train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=50)
#train_dataset = dataset["train"]
eval_dataset = dataset["validation"]
test_dataset = dataset["test"]

In [6]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Define custom function for computing metrics
def compute_metrics(y_pred, y_test):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [7]:
torch.cuda.empty_cache()

In [8]:
from setfit import TrainingArguments

args = TrainingArguments(
    batch_size=16,
    num_epochs=1,
    max_length=512,
)

In [10]:
from setfit import Trainer

for i in [5, 10, 25, 50, 75, 100, 125, 150]:
    print(i)
    model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5") #original retrained model need to be loaded for each round
    train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=i)
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        metric=compute_metrics,
    )
    trainer.train()
    metrics = trainer.evaluate(eval_dataset)
    print(metrics)

5


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 60
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 4


Step,Training Loss


...

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 45300
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 2832


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.8222222222222222, 'precision': 0.8337889557135046, 'recall': 0.8069381598793364, 'f1': 0.812987012987013}


In [9]:
#restart kernel and check 100 sample and 150 sample again to see if the results above is subject to chance or not
from setfit import Trainer

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=100)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics,
)
trainer.train()
metrics = trainer.evaluate(eval_dataset)
print(metrics)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 20200
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 1263


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.8555555555555555, 'precision': 0.8984375, 'recall': 0.8333333333333333, 'f1': 0.8434782608695652}


In [10]:
from setfit import Trainer

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=150)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    metric=compute_metrics,
)
trainer.train()
metrics = trainer.evaluate(eval_dataset)
print(metrics)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 45300
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 2832


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.8222222222222222, 'precision': 0.8337889557135046, 'recall': 0.8069381598793364, 'f1': 0.812987012987013}


In [73]:
#model was overtrained - results not usable
from setfit import Trainer

for i in [5, 10, 25, 50, 75, 100, 125, 150]:
    print(i)
    train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=i)
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        metric=compute_metrics,
    )
    trainer.train()
    metrics = trainer.evaluate(eval_dataset)
    print(metrics)

5


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 60
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 4


Step,Training Loss


***** Running evaluation *****


...

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

***** Running training *****
  Num unique pairs = 45300
  Batch size = 16
  Num epochs = 1
  Total optimization steps = 2832


Step,Training Loss


***** Running evaluation *****


{'accuracy': 0.8888888888888888, 'precision': 0.8995215311004785, 'recall': 0.8778280542986425, 'f1': 0.8842592592592593}


### Hyperparameter Optimization

In [55]:
from setfit import SetFitModel
from typing import Dict, Any

def model_init(params: Dict[str, Any]) -> SetFitModel:
    params = params or {}
    max_iter = params.get("max_iter", 100)
    solver = params.get("solver", "liblinear")
    params = {
        "head_params": {
            "max_iter": max_iter,
            "solver": solver,
        }
    }
    return SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", **params)

In [56]:
from optuna import Trial
from typing import Dict, Union

def hp_space(trial: Trial) -> Dict[str, Union[float, int, str]]:
    return {
        "body_learning_rate": trial.suggest_float("body_learning_rate", 1e-5, 1e-3, log=True),
        "num_epochs": trial.suggest_int("num_epochs", 1, 4),
        "batch_size": trial.suggest_categorical("batch_size", [8, 16, 32, 64]),
        "seed": trial.suggest_int("seed", 1, 42),
        "max_iter": trial.suggest_int("max_iter", 50, 300),
        "solver": trial.suggest_categorical("solver", ["newton-cg", "lbfgs", "liblinear"]),
    }

In [57]:
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=10)

trainer = Trainer(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    model_init=model_init,
    metric=compute_metrics,
)
best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, n_trials=10)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

[I 2024-03-19 15:10:47,767] A new study created in memory with name: no-name-b7270f26-c3be-4c6c-b813-4425cf417960
Trial: {'body_learning_rate': 0.0002234337811022987, 'num_epochs': 2, 'batch_size': 64, 'seed': 16, 'max_iter': 103, 'solver': 'liblinear'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 64
  Num epochs = 2
  Total optimization steps = 8


Step,Training Loss


***** Running evaluation *****
[I 2024-03-19 15:10:56,536] Trial 0 finished with value: 2.910307358698036 and parameters: {'body_learning_rate': 0.0002234337811022987, 'num_epochs': 2, 'batch_size': 64, 'seed': 16, 'max_iter': 103, 'solver': 'liblinear'}. Best is trial 0 with value: 2.910307358698036.
Trial: {'body_learning_rate': 1.1428894284559321e-05, 'num_epochs': 2, 'batch_size': 8, 'seed': 13, 'max_iter': 170, 'solver': 'liblinear'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 8
  Num epochs = 2
  Total optimization steps = 56


...

***** Running evaluation *****
[I 2024-03-19 15:12:14,704] Trial 7 finished with value: 2.9509536827551535 and parameters: {'body_learning_rate': 0.00026605474157681384, 'num_epochs': 2, 'batch_size': 8, 'seed': 18, 'max_iter': 218, 'solver': 'lbfgs'}. Best is trial 2 with value: 3.184867434602662.
Trial: {'body_learning_rate': 1.9907481980999865e-05, 'num_epochs': 2, 'batch_size': 16, 'seed': 40, 'max_iter': 79, 'solver': 'liblinear'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 16
  Num epochs = 2
  Total optimization steps = 28


Step,Training Loss


***** Running evaluation *****
[I 2024-03-19 15:12:23,939] Trial 8 finished with value: 2.857070098822017 and parameters: {'body_learning_rate': 1.9907481980999865e-05, 'num_epochs': 2, 'batch_size': 16, 'seed': 40, 'max_iter': 79, 'solver': 'liblinear'}. Best is trial 2 with value: 3.184867434602662.
Trial: {'body_learning_rate': 2.6655487077965046e-05, 'num_epochs': 1, 'batch_size': 64, 'seed': 33, 'max_iter': 91, 'solver': 'newton-cg'}
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 64
  Num epochs = 1
  Total optimization steps = 4


Step,Training Loss


***** Running evaluation *****
[I 2024-03-19 15:12:28,873] Trial 9 finished with value: 2.727965847166735 and parameters: {'body_learning_rate': 2.6655487077965046e-05, 'num_epochs': 1, 'batch_size': 64, 'seed': 33, 'max_iter': 91, 'solver': 'newton-cg'}. Best is trial 2 with value: 3.184867434602662.


In [58]:
print(best_run)

BestRun(run_id='2', objective=3.184867434602662, hyperparameters={'body_learning_rate': 0.0007601729115127736, 'num_epochs': 3, 'batch_size': 16, 'seed': 20, 'max_iter': 144, 'solver': 'lbfgs'}, backend=<optuna.study.study.Study object at 0x14d9ea795250>)


In [59]:
trainer.apply_hyperparameters(best_run.hyperparameters, final_model=True)
trainer.train()

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
***** Running training *****
  Num unique pairs = 220
  Batch size = 16
  Num epochs = 3
  Total optimization steps = 42


Step,Training Loss


In [60]:
trainer.evaluate(eval_dataset)

***** Running evaluation *****


{'accuracy': 0.8,
 'precision': 0.8162803843979649,
 'recall': 0.7812971342383107,
 'f1': 0.7872899159663864}

In [61]:
trainer.evaluate(test_dataset)

***** Running evaluation *****


{'accuracy': 0.7,
 'precision': 0.7319711538461539,
 'recall': 0.6909945571499257,
 'f1': 0.6827262044653348}

### Making predictions

In [ ]:
preds = model.predict()